# Planeti in lune v fantazijskem svetu Vojne zvezd (Star Wars)

In [13]:
from bs4 import BeautifulSoup
import requests
from googletrans import Translator
import csv
import re

### 1. Pridobivanje podatkov


Najprej definiramo spremenljivke, ki jih bomo potrebovali tekom naše ekstrakcije podatkov:

In [36]:
# Povezava do spletne strani
planeti_lune_url_1 = 'https://en.wikipedia.org/wiki/List_of_Star_Wars_planets_and_moons'
planeti_lune_url_2 = 'https://starwars.fandom.com/wiki/List_of_planets'

# Html izbrane strani
planeti_lune_html_1 = 'Planeti_in_lune.html'
planeti_lune_html_2 = 'List_of_planets.html'

# Imena csv datotek
planeti_lune_csv_1 = 'planeti-lune-1.csv'
planeti_lune_csv_2 = 'planeti-lune-2.csv'

Nato pa še funkcije:

In [8]:
def prevedi_v_slovenščino(besedilo):
    prevajalnik = Translator()
    prevod = prevajalnik.translate(besedilo, src='en', dest='sl') # Iz angleščine v slovenščino
    return prevod.text

In [9]:
def pridobi_podatke(planeti_lune_url):
    try:
        response = requests.get(planeti_lune_url)
        if response.status_code == 200: # 200 pomeni, da je vse okej
            return response.text
        else: 
            raise ValueError(f'Napačna koda: {response.status_code}')
    except Exception: # Napaka
        print(f'Nalaganje strani {planeti_lune_url} ni uspelo.')

In [10]:
def odstrani_oklepaje(text):
    očiščeno = re.sub(r'\[\d+\]', '', text)
    return očiščeno.strip()

### 2. Luščenje podatkov

Najprej za prvo stran (Wikipedija):

In [37]:
def luščenje_podatkov_1(planeti_lune_url_1):

    odziv = requests.get(planeti_lune_url_1)
    html_podatki = odziv.content
    juhica = BeautifulSoup(html_podatki, 'html.parser') # Razčlenimo
    tabela = juhica.find('table', class_='wikitable sortable')  # Tabela v Wikipediji

    podatki_wiki = []  # V ta slovar bomo zapisali izluščene podatke

    # Vrstice
    vrstice = tabela.find_all('tr')
    for vrstica in vrstice[1:]:  # Izpustimo glavo
        okvirček = vrstica.find_all('td')
        if len(okvirček) >= 5:  
            ime = okvirček[0].get_text(strip=True) # Ime planeta oz. lune
            prvič = okvirček[1].get_text(strip=True)
            leto = okvirček[2].get_text(strip=True)
            description = okvirček[4].get_text(strip=True)

            ali_je_planet = "planet" in description.lower()  # Preveri, če se koren planet pojavi v opisu
            vrsta = "planet" if ali_je_planet else "luna"  # Če se ne pojavi, izpiše luna

            prevedeno = prevedi_v_slovenščino(description) # Prevod v slovenščino

            podatki_wiki.append({   
                'Ime': ime,
                'Vrsta': vrsta,
                'Prva upodobitev': prvič,
                'Leto prve upodobitve': leto,
                'Kratek opis': prevedeno,     
            })
        
    return podatki_wiki 

Zdaj še preiskusimo delovanje:

In [38]:
# Test
podatki_1 = luščenje_podatkov_1(planeti_lune_url_1)

for element in podatki_1:
    print(element)

{'Ime': 'Abafar', 'Vrsta': 'planet', 'Prva upodobitev': 'Star Wars: The Clone Wars', 'Leto prve upodobitve': '2012', 'Kratek opis': 'Puščavski planet, ki se nahaja v zunanjem robu s popolnoma belo površino.Planet, znan kot Praznina, je komaj poseljen, vendar je dom ogromnim količinam ridonija, redkega in hlapljivega goriva.'}
{'Ime': 'Agamar', 'Vrsta': 'planet', 'Prva upodobitev': 'Star Wars Legends:The Farlander Papers', 'Leto prve upodobitve': '1993', 'Kratek opis': 'Pust, skalnat planet v zunanjem robu.Mesto še vedno aktivnega rezervata separatističnih droidov, ki so postali znani po tem, da so preživeli vojne klonov.'}
{'Ime': 'Ahch-To', 'Vrsta': 'planet', 'Prva upodobitev': 'The Force Awakens', 'Leto prve upodobitve': '2015', 'Kratek opis': 'Velik oceanski planet, na katerem se že več let skriva Luke Skywalker, in lokacija prvega jedijevskega templja in drevesne knjižnice svetih jedijskih besedil. Porgi, lanai in thala-sirene so med domorodnimi vrstami.'}
{'Ime': 'Ajan Kloss', 'Vr

Zdaj pa še za drugo spletno stran (Wookipedia):

In [39]:
def luščenje_podatkov_2(planeti_lune_url_2):
    odziv = requests.get(planeti_lune_url_2)
    html_podatki = odziv.content
    juhica = BeautifulSoup(html_podatki, 'html.parser')

    podatki_wooki = []  # To store the extracted data

    # Find all tables with planet data
    tabele = juhica.find_all('table', class_='wikitable')
    for tabela in tabele: # Še ena zanka ker imamo več tabel
        vrstice = tabela.find_all('tr')
        for vrstica in vrstice[1:]:  # Odstranimo glavo
            celice = vrstica.find_all('td')
            if len(celice) >= 5:  # Mora biti doovolj celic
                ime = odstrani_oklepaje(celice[0].get_text(strip=True)) 
                regija = odstrani_oklepaje(celice[1].get_text(strip=True))
                sektor = odstrani_oklepaje(celice[2].get_text(strip=True)) 
                sistem = odstrani_oklepaje(celice[3].get_text(strip=True)) 
                prebivalstvo = odstrani_oklepaje(celice[4].get_text(strip=True))
    
                if ime != 'Unknown' or regija != 'Unknown' or sektor != 'Unknown' or sistem != 'Unknown' or prebivalstvo != 'Unknown':
                    podatki_wooki.append({
                        'Ime': ime,
                        'Regija': regija,
                        'Sektor': sektor,
                        'Sistem': sistem,
                        'Prebivalstvo': prebivalstvo
                    })

    return podatki_wooki

Ponovno potestiramo:

In [40]:
# Test
podatki_2 = luščenje_podatkov_2(planeti_lune_url_2)

for element in podatki_2:
    print(element)

{'Ime': '5251977', 'Regija': 'Unknown', 'Sektor': 'Unknown', 'Sistem': 'Unknown', 'Prebivalstvo': 'Unknown'}
{'Ime': 'Aaloth', 'Regija': 'Outer Rim Territories', 'Sektor': 'Gaulus sector', 'Sistem': 'Unknown', 'Prebivalstvo': "Twi'leks"}
{'Ime': 'Aargonar', 'Regija': 'Unknown', 'Sektor': 'Unknown', 'Sistem': 'Unknown', 'Prebivalstvo': 'Humans'}
{'Ime': 'Ab Dalis', 'Regija': 'Outer Rim Territories', 'Sektor': 'Unknown', 'Sistem': 'Ab Dalis system', 'Prebivalstvo': 'Unknown'}
{'Ime': 'Abafar', 'Regija': 'Outer Rim Territories', 'Sektor': 'Sprizen sector', 'Sistem': 'Abafar system', 'Prebivalstvo': 'Various'}
{'Ime': 'Abednedo', 'Regija': 'Colonies', 'Sektor': 'Unknown', 'Sistem': 'Unknown', 'Prebivalstvo': 'Abednedo'}
{'Ime': 'Abelor', 'Regija': 'Mid Rim Territories', 'Sektor': 'Unknown', 'Sistem': 'Unknown', 'Prebivalstvo': 'Unknown'}
{'Ime': 'Abregado-rae', 'Regija': 'Core Worlds', 'Sektor': 'Unknown', 'Sistem': 'Abregado system', 'Prebivalstvo': 'Unknown'}
{'Ime': 'Absanz', 'Regija': 

### 3. Shranjevanje podatkov

Najprej v csv format shranimo prvo pridobljene podatke:

In [24]:
def shrani_v_csv_1(podatki, csv_file):
    stolpci = ['Ime', 'Vrsta', 'Prva upodobitev', 'Leto prve upodobitve', 'Kratek opis',]

    with open(csv_file, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=stolpci)
        writer.writeheader()
        writer.writerows(podatki_1)
        for entry in podatki:
            writer.writerow(entry)

    print('Podatki prepisani na', csv_file)

In [27]:
# Prepišemo
shrani_v_csv_1(podatki_1, planeti_lune_csv_1)

Podatki prepisani na planeti-lune-1.csv


Nato pa še druge:

In [41]:
def shrani_v_csv_2(podatki, csv_file):
    stolpci = ['Ime', 'Regija', 'Sektor', 'Sistem', 'Prebivalstvo',]

    with open(csv_file, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=stolpci)
        writer.writeheader()
        writer.writerows(podatki_2)
        for entry in podatki:
            writer.writerow(entry)

    print('Podatki prepisani na', csv_file)

In [34]:
# Prepišemo
shrani_v_csv_2(podatki_2, planeti_lune_csv_2)

Podatki prepisani na planeti-lune-2.csv
